**CONAD**

In [ ]:
# Libraries
import pandas as pd
import re
## Importo il dataset
conad = pd.read_csv('https://raw.githubusercontent.com/CoroTheBoss/DM-project/main/conad_final.csv')
conad = conad.drop(['unit_price'], axis=1)
conad = conad.rename(columns={'products': 'Product', 'prices_per_unitmeasurement': 'Price','category': 'Category', 'subcategory': 'Subcategory'})
conad.head()

,Product,Price,Category,Subcategory
0,"Arance Valencia Italia cal. 7 1,5 kg Percorso ...","2,65 € / Kg",Frutta e verdura,Frutta fresca
1,Pompelmi Rossi Cal. 4/5 Cat. I,"2,48 € / Kg",Frutta e verdura,Frutta fresca
2,Limone Costa d'Amalfi I.G.P. Primofiore Italia...,"5,56 € / Kg",Frutta e verdura,Frutta fresca
3,"Limoni Verdello Italia 0,500 kg Verso Natura C...","3,96 € / Kg",Frutta e verdura,Frutta fresca
4,Percorso Qualità Limoni Italia Cal. 4 Cat. I R...,"3,96 € / Kg",Frutta e verdura,Frutta fresca


In [ ]:
## Estraggo le quantità dei prodotti
pattern = r'((?:\d+,\d+|\d+|\d+\s*x\s*(?:\d+,\d+|\d+\.?\d+))\s*(?:g|kg|Kg|l|L|ml|ML|pz|ml|gr|buste|fogli|guanti|paio|pezzi|salviette|sacchi|m|cm|cl|litro|uova)\b)'
conad['Quantity'] = conad['Product'].apply(lambda x: re.findall(pattern, x)[-1] if len(re.findall(pattern, x)) > 0 else None)
conad.head()

,Product,Price,Category,Subcategory,Quantity
0,"Arance Valencia Italia cal. 7 1,5 kg Percorso ...","2,65 € / Kg",Frutta e verdura,Frutta fresca,"1,5 kg"
1,Pompelmi Rossi Cal. 4/5 Cat. I,"2,48 € / Kg",Frutta e verdura,Frutta fresca,None
2,Limone Costa d'Amalfi I.G.P. Primofiore Italia...,"5,56 € / Kg",Frutta e verdura,Frutta fresca,500 g
3,"Limoni Verdello Italia 0,500 kg Verso Natura C...","3,96 € / Kg",Frutta e verdura,Frutta fresca,"0,500 kg"
4,Percorso Qualità Limoni Italia Cal. 4 Cat. I R...,"3,96 € / Kg",Frutta e verdura,Frutta fresca,500g


In [ ]:
## Stampo il set di unità di misura presenti nel dataset e inizio a uniformare
conad['Quantity'] = conad['Quantity'].str.replace('buste', 'pz')
conad['Quantity'] = conad['Quantity'].str.replace('fogli', 'pz')
conad['Quantity'] = conad['Quantity'].str.replace('guanti', 'pz')
conad['Quantity'] = conad['Quantity'].str.replace('paio', 'pz')
conad['Quantity'] = conad['Quantity'].str.replace('pezzi', 'pz')
conad['Quantity'] = conad['Quantity'].str.replace('sacchi', 'pz')
conad['Quantity'] = conad['Quantity'].str.replace('salviette', 'pz')
conad['Quantity'] = conad['Quantity'].str.replace('uova', 'pz')
conad['Quantity'] = conad['Quantity'].str.replace('gr', 'g')
conad['Quantity'] = conad['Quantity'].str.replace('litro', 'l')
## Sostituisco virgola con punto (serve dopo per convertire in float)
conad['Quantity'] = conad['Quantity'].str.replace(',', '.')

In [ ]:
## Creo funzione che esegua le moltiplicazioni
def process_quantity(s):
    if s is None:
        return None
    # Aggiungi spazio tra numero e lettere se non c'è
    s = re.sub(r'(\d)([a-zA-Z])', r'\1 \2', s)
    # Risolvi la moltiplicazione se presente
    if 'x' in s:
        parts = s.split('x')
        result = float(parts[0].strip()) * float(parts[1].split()[0].strip())
        unit = parts[1].split()[1]
        return str(result) + " " + unit
    else:
        return s

In [ ]:
# Applicazione funzione moltiplicazione con il metodo .apply()
conad['Quantity_processed'] = conad['Quantity'].apply(process_quantity)
conad.head()

,Product,Price,Category,Subcategory,Quantity,Quantity_processed
0,"Arance Valencia Italia cal. 7 1,5 kg Percorso ...","2,65 € / Kg",Frutta e verdura,Frutta fresca,1.5 kg,1.5 kg
1,Pompelmi Rossi Cal. 4/5 Cat. I,"2,48 € / Kg",Frutta e verdura,Frutta fresca,None,None
2,Limone Costa d'Amalfi I.G.P. Primofiore Italia...,"5,56 € / Kg",Frutta e verdura,Frutta fresca,500 g,500 g
3,"Limoni Verdello Italia 0,500 kg Verso Natura C...","3,96 € / Kg",Frutta e verdura,Frutta fresca,0.500 kg,0.500 kg
4,Percorso Qualità Limoni Italia Cal. 4 Cat. I R...,"3,96 € / Kg",Frutta e verdura,Frutta fresca,500g,500 g


In [ ]:
## Creo colonne con quantità e unità di misura
conad[['Quantity_num_giusto', 'Quantity_unità di misura']] = conad['Quantity_processed'].str.rsplit(' ', 1, expand=True)
# Leggo numeri come float e sostituisco virgola con punto
conad['Quantity_num_giusto'] = conad['Quantity_num_giusto'].apply(lambda x: float(x.replace(',','.')) if x is not None else x)

In [ ]:
## Converto le unità di misura
def convert_to_SI(row):
    unit = row['Quantity_unità di misura']
    value = row['Quantity_num_giusto']

    if unit == 'g':
        return value / 1000, 'kg'  # convert grams to kilograms
    elif unit == 'ml':
        return value / 1000, 'l'  # convert milliliters to liters
    elif unit in ['kg', 'l', 'pz']:  # keep these as they are
        return value, unit
    else:
        return value, unit  # if you want to keep other units as they are
conad[['Quantity_SI', 'Quantity_unità di misura_SI']] = conad.apply(convert_to_SI, axis=1, result_type='expand')
conad.head()

,Product,Price,Category,Subcategory,Quantity,Quantity_processed,Quantity_num_giusto,Quantity_unità di misura,Quantity_SI,Quantity_unità di misura_SI
0,"Arance Valencia Italia cal. 7 1,5 kg Percorso ...","2,65 € / Kg",Frutta e verdura,Frutta fresca,1.5 kg,1.5 kg,1.5,kg,1.5,kg
1,Pompelmi Rossi Cal. 4/5 Cat. I,"2,48 € / Kg",Frutta e verdura,Frutta fresca,None,None,NaN,None,NaN,NaN
2,Limone Costa d'Amalfi I.G.P. Primofiore Italia...,"5,56 € / Kg",Frutta e verdura,Frutta fresca,500 g,500 g,500.0,g,0.5,kg
3,"Limoni Verdello Italia 0,500 kg Verso Natura C...","3,96 € / Kg",Frutta e verdura,Frutta fresca,0.500 kg,0.500 kg,0.5,kg,0.5,kg
4,Percorso Qualità Limoni Italia Cal. 4 Cat. I R...,"3,96 € / Kg",Frutta e verdura,Frutta fresca,500g,500 g,500.0,g,0.5,kg


In [ ]:
## Unisco le nuove colonne  ed elimino quelle non più necessarie
conad['Quantity_final'] = conad['Quantity_SI'].astype(str) + " " + conad['Quantity_unità di misura_SI']
conad = conad.drop('Quantity_unità di misura_SI', axis = 1)
conad = conad.drop('Quantity_num_giusto', axis = 1)
conad = conad.drop('Quantity_SI', axis = 1)
conad = conad.drop('Quantity_unità di misura', axis = 1)
conad = conad.drop('Quantity_processed', axis = 1)
conad = conad.drop(['Quantity', 'Subcategory'], axis = 1)
conad['Price'] = conad['Price'].str.replace(',','.')
conad = conad.rename(columns={'Quantity_final':'Quantity'})
conad.head()

,Product,Price,Category,Quantity
0,"Arance Valencia Italia cal. 7 1,5 kg Percorso ...",2.65 € / Kg,Frutta e verdura,1.5 kg
1,Pompelmi Rossi Cal. 4/5 Cat. I,2.48 € / Kg,Frutta e verdura,NaN
2,Limone Costa d'Amalfi I.G.P. Primofiore Italia...,5.56 € / Kg,Frutta e verdura,0.5 kg
3,"Limoni Verdello Italia 0,500 kg Verso Natura C...",3.96 € / Kg,Frutta e verdura,0.5 kg
4,Percorso Qualità Limoni Italia Cal. 4 Cat. I R...,3.96 € / Kg,Frutta e verdura,0.5 kg


uniformo colonne

In [ ]:
# Deleting duplicates
conad = conad.drop_duplicates(subset=['Product'], keep='first')
conad = conad.reset_index()

**CARREFOUR**

In [ ]:
cols = ['Product','Price','Quantity','Category']
carrefour = pd.read_csv('https://raw.githubusercontent.com/CoroTheBoss/DM-project/main/carrefour_final.csv', index_col=0)
carrefour.head()

,Product,unit Price,Category,Unit Price [€],Unit Type,Quantity [units],Quantity,Unit of measure,Product Price [€]
0,Carrefour Correttore a nastro maxi 5x12 mm,"€ 1,89 al\npz/1.0 pz",articoli per la casa,1.89,pz,1.0 pz,1.0,pz,1.89
1,Carrefour Confezione di 4 Penne a Sfera Blu Pe...,"€ 1,49 al\npz/1.0 pz",articoli per la casa,1.49,pz,1.0 pz,1.0,pz,1.49
2,Carrefour Forbici punte arrotondate 17cm,"€ 2,99 al\npz/1.0 pz",articoli per la casa,2.99,pz,1.0 pz,1.0,pz,2.99
3,Carrefour Confezione di 4 Penne a Sfera Nero P...,"€ 1,49 al\npz/1.0 pz",articoli per la casa,1.49,pz,1.0 pz,1.0,pz,1.49
4,Carrefour Quaderno A4 punto metallico con cope...,"€ 1,59 al\npz/1.0 pz",articoli per la casa,1.59,pz,1.0 pz,1.0,pz,1.59


In [ ]:
## funzioni
def format_price_unit_2(row):
    row = str(row)
    parts = row.split()
    prezzo = parts[1]
    unita = parts[3]
    return f"{prezzo} € / {unita}"
def get_quantity_carrefour(row):
    row = str(row)
    parts = row.split()
    quantity = parts[-2]
    unita = parts[-1]
    return f"{quantity} {unita}"
def format_quantity_carrefour(row):
    row = str(row)
    parts = row.split()
    if 'g' in parts:
        parts[0] = float(parts[0])/1000
        parts[1] = 'kg'
    if 'ml' in parts:
        parts[0] = float(parts[0])/1000
        parts[1] = 'l'
    if 'gr' in parts:
        parts[0] = float(parts[0])/1000
        parts[1] = 'kg'
    if 'cl' in parts:
        parts[0] = float(parts[0])/100
        parts[1] = 'l'
    out = str(parts[0]) + ' ' +  parts[1]
    return out

In [ ]:
# Dropping unwanted columns, resetting index and applying functions
carrefour = carrefour.reset_index(drop=True)
carrefour = carrefour.drop(['Unit Price [€]'], axis=1)
carrefour = carrefour.drop(['Unit Type'], axis=1)
carrefour = carrefour.drop(['Quantity [units]'], axis=1)
carrefour = carrefour.drop(['Quantity'], axis=1)
carrefour = carrefour.drop(['Unit of measure'], axis=1)
carrefour = carrefour.drop(['Product Price [€]'], axis=1)
carrefour['unit Price'] = carrefour['unit Price'].str.replace('\n',' ')
carrefour['unit Price'] = carrefour['unit Price'].str.replace('/',' ')
carrefour = carrefour.rename(columns={'unit Price': 'Price'})
carrefour['Quantity'] = carrefour['Price'].apply(get_quantity_carrefour)
carrefour['Quantity'] = carrefour['Quantity'].apply(format_quantity_carrefour)
# Lowering strings
carrefour['Product'] = carrefour['Product'].str.lower()
carrefour['Category'] = carrefour['Category'].str.lower()
carrefour['Price'] = carrefour['Price'].apply(format_price_unit_2)
# Replacing comma
carrefour['Price'] = carrefour['Price'].str.replace(',','.')
carrefour = carrefour[cols]
carrefour.head()

,Product,Price,Quantity,Category
0,carrefour correttore a nastro maxi 5x12 mm,1.89 € / pz,1.0 pz,articoli per la casa
1,carrefour confezione di 4 penne a sfera blu pe...,1.49 € / pz,1.0 pz,articoli per la casa
2,carrefour forbici punte arrotondate 17cm,2.99 € / pz,1.0 pz,articoli per la casa
3,carrefour confezione di 4 penne a sfera nero p...,1.49 € / pz,1.0 pz,articoli per la casa
4,carrefour quaderno a4 punto metallico con cope...,1.59 € / pz,1.0 pz,articoli per la casa


In [ ]:
# Deleting duplicates
carrefour = carrefour.drop_duplicates(subset=['Product'], keep='first')
carrefour = carrefour.reset_index()

**ESSELUNGA**

In [ ]:
# Reading file from GitHub
esselunga = pd.read_csv("https://raw.githubusercontent.com/CoroTheBoss/DM-project/main/esselunga_final.csv", index_col=0)
esselunga.head()

,Category,Product,Price
0,SPESA BIO,"Esselunga Bio, cipolle dorate 500 g","Euro 3,16 / kg"
1,SPESA BIO,"Esselunga Bio, datterini biologici 300 g","Euro 8,27 / kg"
2,SPESA BIO,"Esselunga Bio, Insalatina 100 g","Euro 14,80 / kg"
3,SPESA BIO,"Esselunga Bio, 6 uova fresche biologiche","Euro 0,49 / pz"
4,SPESA BIO,"Esselunga Bio, prezzemolo tritato biologico su...","Euro 23,00 / kg"


In [ ]:
## Tutti i prezzi nello stesso formato
def format_price_unit(row):
    row = str(row)
    parts = row.split("/")
    prezzo_part = parts[0].split()
    unita_part = parts[1].split()
    if "€" in prezzo_part:
        prezzo = prezzo_part[0]
    if "Euro" in prezzo_part:
        prezzo = prezzo_part[1]
    unita = unita_part[0]
    return f"{prezzo} € / {unita}"

In [ ]:
esselunga = esselunga.dropna(subset=['Price'])
esselunga['Price'] = esselunga['Price'].apply(format_price_unit)
## 3. Tutto in minuscolo 
esselunga['Product'] = esselunga['Product'].str.lower()
esselunga['Category'] = esselunga['Category'].str.lower()

In [ ]:
## 4. Estrazione quantità
pattern = r'((?:\d+,\d+|\d+|\d+\s*x\s*(?:\d+,\d+|\d+\.?\d+))\s*(?:g|kg|Kg|l|L|ml|ML|pz|ml|gr|buste|fogli|guanti|paio|pezzi|salviette|sacchi|m|cm|cl|litro|uova)\b)'
# Applying pattern
esselunga['Quantity'] = esselunga['Product'].apply(lambda x: re.findall(pattern, x)[-1] if len(re.findall(pattern, x)) > 0 else None)
### Removing unwanted unit measurements
esselunga['Quantity'] = esselunga['Quantity'].apply(lambda x: None if 'cm' in str(x) else x)
esselunga['Quantity'] = esselunga['Quantity'].apply(lambda x: None if 'mm' in str(x) else x)
esselunga['Quantity'] = esselunga['Quantity'].apply(lambda x: None if 'm' in str(x) else x)
### Uniforming quantities
esselunga['Quantity'] = esselunga['Quantity'].str.replace('pezzi', 'pz') 
esselunga['Quantity'] = esselunga['Quantity'].str.replace('Kg', 'kg') 
esselunga['Quantity'] = esselunga['Quantity'].str.replace('L', 'l') 
esselunga['Quantity'] = esselunga['Quantity'].str.replace('ML', 'ml')
esselunga['Quantity'] = esselunga['Quantity'].str.replace('buste', 'pz')
esselunga['Quantity'] = esselunga['Quantity'].str.replace('fogli', 'pz')
esselunga['Quantity'] = esselunga['Quantity'].str.replace('Ml', 'ml')
esselunga['Quantity'] = esselunga['Quantity'].str.replace('gr', 'g')
esselunga['Quantity'] = esselunga['Quantity'].str.replace('fogli', 'pz')
esselunga['Quantity'] = esselunga['Quantity'].str.replace('paio', 'pz')
esselunga['Quantity'] = esselunga['Quantity'].str.replace('sacchi', 'pz')
esselunga['Quantity'] = esselunga['Quantity'].str.replace('uova', 'pz')
esselunga['Quantity'] = esselunga['Quantity'].str.replace('sacchi', 'pz')
# Replacing comma
esselunga['Quantity'] = esselunga['Quantity'].str.replace(',','.')

In [ ]:
# Applico funzione process_quantity definita prima
esselunga['Quantity_processed'] = esselunga['Quantity'].apply(process_quantity)
esselunga.head()
esselunga[['Quantity_num_giusto', 'Quantity_unità di misura']] = esselunga['Quantity_processed'].str.rsplit(' ', 1, expand=True)
esselunga.head()
esselunga['Quantity_num_giusto'] = esselunga['Quantity_num_giusto'].apply(lambda x: float(x.replace(',','.')) if x is not None else x)

In [ ]:
### definisco funzione per conversione unità di misura in: kg, l, pz
def convert_to_SI(row):
    unit = row['Quantity_unità di misura']
    value = row['Quantity_num_giusto']

    if unit == 'g':
        return value / 1000, 'kg'  # convert grams to kilograms
    elif unit == 'ml':
        return value / 1000, 'l'  # convert milliliters to liters
    elif unit in ['kg', 'l', 'pz']:  # keep these as they are
        return value, unit
    else:
        return value, unit

In [ ]:
esselunga[['Quantity_SI', 'Quantity_unità di misura_SI']] = esselunga.apply(convert_to_SI, axis=1, result_type='expand')
esselunga.head()
### unifico le colonne e elimino quelle che non servono
esselunga['Quantity_final'] = esselunga['Quantity_SI'].astype(str) + " " + esselunga['Quantity_unità di misura_SI']
esselunga = esselunga.drop('Quantity_unità di misura_SI', axis = 1)
esselunga = esselunga.drop('Quantity_num_giusto', axis = 1)
esselunga = esselunga.drop('Quantity_SI', axis = 1)
esselunga = esselunga.drop('Quantity_unità di misura', axis = 1)
esselunga = esselunga.drop('Quantity_processed', axis = 1)
esselunga = esselunga.drop('Quantity', axis = 1)
esselunga = esselunga.rename(columns={'Quantity_final':'Quantity'})
# Replacing comma
esselunga['Price'] = esselunga['Price'].str.replace(',','.')
esselunga.head()

,Category,Product,Price,Quantity
0,spesa bio,"esselunga bio, cipolle dorate 500 g",3.16 € / kg,0.5 kg
1,spesa bio,"esselunga bio, datterini biologici 300 g",8.27 € / kg,0.3 kg
2,spesa bio,"esselunga bio, insalatina 100 g",14.80 € / kg,0.1 kg
3,spesa bio,"esselunga bio, 6 uova fresche biologiche",0.49 € / pz,6.0 pz
4,spesa bio,"esselunga bio, prezzemolo tritato biologico su...",23.00 € / kg,0.05 kg


uniformo colonne

In [ ]:
## Risolvo problema dei detersivi per esselunga (prezzo = €/misurino)
def adjust_detergents(row):
    if 'ms' in row['Price']:
        price = float(re.search('(\d+\.\d+)', row['Price']).group())
        quantity = float(re.search('(\d+)', row['Product']).group())
        price *= quantity
        return f'{price:.2f} € / pz'
    else:
        return row['Price']

In [ ]:
esselunga['Price'] = esselunga.apply(adjust_detergents, axis=1)

In [ ]:
esselunga.head()

,Category,Product,Price,Quantity
0,spesa bio,"esselunga bio, cipolle dorate 500 g",3.16 € / kg,0.5 kg
1,spesa bio,"esselunga bio, datterini biologici 300 g",8.27 € / kg,0.3 kg
2,spesa bio,"esselunga bio, insalatina 100 g",14.80 € / kg,0.1 kg
3,spesa bio,"esselunga bio, 6 uova fresche biologiche",0.49 € / pz,6.0 pz
4,spesa bio,"esselunga bio, prezzemolo tritato biologico su...",23.00 € / kg,0.05 kg


In [ ]:
# Deleting duplicates
esselunga = esselunga.drop_duplicates(subset=['Product'], keep='first')
esselunga = esselunga.reset_index()

**SAVING CSV**

In [ ]:
## Salvo csv
#esselunga.to_csv('esselunga_preprocessed.csv', index=False)
#carrefour.to_csv('carrefour_preprocessed.csv', index=False)
#conad.to_csv('conad_preprocessed.csv', index=False)